In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Simple_Linear_Regression').getOrCreate()

In [2]:
df = spark.read.csv('/FileStore/tables/Ecommerce_Customers.csv', inferSchema='True', header='True')

In [3]:
df.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
 Email| Address| Avatar|Avg Session Length| Time on App| Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
mstephenson@ferna...|835 Frank TunnelW...| Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616| 4.0826206329529615| 587.9510539684005|
 hduke@hotmail.com|4547 Archer Commo...| DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744| 2.66403418213262| 392.2049334443264|
 pallen@yahoo.com|24645 Valerie Uni...| Bisque|33.000914755642675|11.330278057777512|37.110597442120856| 4.104543202376424| 487.54750486747207|
riverarebecca@gma...|1414 David Throug...| SaddleBrown| 34.30555662975554|13.717513665142507| 36.72128267790313| 3.120178782748092| 581.8523440352177|
mstephens@davidso...|14023 Rodriguez P...|MediumAquaMarine| 33.33067252364639|12.795188551078114| 37.53665330059473| 4.446308318351434| 599.4060920457634|
alvareznancy@luca...|645 Martha Park A...| FloralWhite|33.871037879341976|12.026925339755056| 34.47687762925054| 5.493507201364199| 637.102447915074|
katherine20@yahoo...|68388 Reyes Light...| DarkSlateBlue| 32.02159550138701|11.366348309710526| 36.68377615286961| 4.685017246570912| 521.5721747578274|
 awatkins@yahoo.com|Unit 6538 Box 898...| Aqua|32.739142938380326| 12.35195897300293| 37.37335885854755| 4.4342734348999375| 549.9041461052942|
vchurch@walter-ma...|860 Lee KeyWest D...| Salmon| 33.98777289568564|13.386235275676436|37.534497341555735| 3.2734335777477144| 570.2004089636196|
 bonnie69@lin.biz|PSC 2734, Box 525...| Brown|31.936548618448917|11.814128294972196| 37.14516822352819| 3.202806071553459| 427.1993848953282|
andrew06@peterson...|26104 Alexander G...| Tomato|33.992572774953736|13.338975447662113| 37.22580613162114| 2.482607770510596| 492.6060127179966|
ryanwerner@freema...|Unit 2413 Box 034...| Tomato| 33.87936082480498|11.584782999535266| 37.08792607098381| 3.71320920294043| 522.3374046069357|
 knelson@gmail.com|6705 Miller Orcha...| RoyalBlue|29.532428967057943|10.961298400154098| 37.42021557502538| 4.046423164299585| 408.6403510726275|
wrightpeter@yahoo...|05302 Dunlap Ferr...| Bisque| 33.19033404372265|12.959226091609382|36.144666700041924| 3.918541839158999| 573.4158673313865|
taylormason@gmail...|7773 Powell Sprin...| DarkBlue|32.387975853153876|13.148725692056516| 36.61995708279922| 2.494543646659249| 470.4527333009554|
 jstark@anderson.com|49558 Ramirez Roa...| Peru|30.737720372628182|12.636606052000127|36.213763093698624| 3.3578468423262944| 461.7807421962299|
 wjennings@gmail.com|6362 Wilson Mount...| PowderBlue| 32.12538689728784|11.733861690857394| 34.8940927514398| 3.1361327164897803| 457.84769594494855|
rebecca45@hale-ba...|8982 Burton RowWi...| OliveDrab|32.338899323067196|12.013194694014402| 38.38513659413844| 2.420806160901484| 407.70454754954415|
alejandro75@hotma...|64475 Andre Club ...| Cyan|32.187812045932155| 14.7153875441565| 38.24411459434352| 1.516575580831944| 452.3156754800354|
samuel46@love-wes...|544 Alexander Hei...| LightSeaGreen| 32.61785606282345|13.989592555825254|37.190503800397956| 4.064548550437977| 605.061038804892|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
only showing top 20 rows

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [5]:
df.columns[3:-1]

Out[10]: ['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership']

In [6]:
assembler = VectorAssembler(inputCols=df.columns[3:-1],
                            outputCol='features')

In [7]:
final_df = assembler.transform(df).select('features', 'Yearly Amount Spent')

In [8]:
train_data, test_data = final_df.randomSplit([0.7, 0.3])

In [9]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 334|
 mean| 498.3599823489529|
 stddev| 75.95786049817511|
 min| 275.9184206503857|
 max| 765.5184619388373|
+-------+-------------------+

In [10]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 166|
 mean| 501.23364472737995|
 stddev| 85.87994041445664|
 min| 256.67058229005585|
 max| 725.5848140556806|
+-------+-------------------+

In [11]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent', predictionCol='prediction')
lr = lr.fit(train_data)

In [12]:
test_results = lr.evaluate(test_data)

In [13]:
test_results.rootMeanSquaredError

Out[50]: 10.702717063892337

In [14]:
test_results.r2

Out[51]: 0.984374685616886

In [15]:
final_df.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [16]:
unlabeled_data = test_data.select('features')

In [17]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[30.3931845423455...|
[30.5743636841713...|
[30.7377203726281...|
[30.8162006488763...|
[31.0472221394875...|
[31.1280900496166...|
[31.3123495994443...|
[31.3584771924370...|
[31.3662121671876...|
[31.3895854806643...|
[31.5147378578019...|
[31.6005122003032...|
[31.6098395733896...|
[31.6253601348306...|
[31.6548096756927...|
[31.6739155032749...|
[31.7656188210424...|
[31.8279790554652...|
[31.8512531286083...|
[31.8745516945853...|
+--------------------+
only showing top 20 rows

In [18]:
predictions = lr.transform(unlabeled_data)

In [19]:
test_data.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[30.3931845423455...| 319.9288698031936|
[30.5743636841713...| 442.06441375806565|
[30.7377203726281...| 461.7807421962299|
[30.8162006488763...| 266.086340948469|
[31.0472221394875...| 392.4973991890214|
[31.1280900496166...| 557.2526867470547|
[31.3123495994443...| 463.5914180279406|
[31.3584771924370...| 495.1759504494754|
[31.3662121671876...| 430.5888825564849|
[31.3895854806643...| 410.0696110599829|
[31.5147378578019...| 489.8124879964614|
[31.6005122003032...| 479.17285149109694|
[31.6098395733896...| 444.54554965110816|
[31.6253601348306...| 376.3369007569242|
[31.6548096756927...| 475.2634237275485|
[31.6739155032749...| 475.7250679098812|
[31.7656188210424...| 496.55408163560713|
[31.8279790554652...| 440.0027475469415|
[31.8512531286083...| 472.9922466667984|
[31.8745516945853...| 392.2852442462675|
+--------------------+-------------------+
only showing top 20 rows

In [20]:
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[30.3931845423455...|330.32330437021506|
[30.5743636841713...| 440.9325942603555|
[30.7377203726281...|449.75377488699155|
[30.8162006488763...| 282.5128432474394|
[31.0472221394875...|388.39891171062936|
[31.1280900496166...| 564.0606692169422|
[31.3123495994443...| 445.1954197188554|
[31.3584771924370...| 490.1967218241041|
[31.3662121671876...|426.11649022562233|
[31.3895854806643...|409.53647261652327|
[31.5147378578019...| 496.4436785405967|
[31.6005122003032...| 460.3433256036453|
[31.6098395733896...|426.41959314924884|
[31.6253601348306...| 380.7042249461581|
[31.6548096756927...| 468.6996988071528|
[31.6739155032749...| 501.5481421853867|
[31.7656188210424...|501.41747903665396|
[31.8279790554652...|449.16784601228915|
[31.8512531286083...|463.50084800116247|
[31.8745516945853...| 397.3884732568806|
+--------------------+------------------+
only showing top 20 rows